In [1]:
import os
# to use or not to use GPU
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

# packages for model graph visualization
#!pip install -q pydot
# install graphviz https://graphviz.gitlab.io/download/ 

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub
!pip install -q -U tensorflow-text
import tensorflow_text as text
!pip install -q tf-models-official
from official.nlp import optimization  # to create AdamW optimizer

import json
import re
import random
import numpy as np
import matplotlib.pyplot as plt

#save_path = "/content/drive/My Drive/Colab Notebooks/"

     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 42.5 MB/s 
     |████████████████████████████████| 213 kB 50.3 MB/s 
     |████████████████████████████████| 90 kB 9.1 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 47.7 MB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 352 kB 54.3 MB/s 
     |████████████████████████████████| 99 kB 10.2 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
save_path = "/content/drive/My Drive/Colab Notebooks/"

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# checking if we have access to a GPU

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
#@title Data Loading & Preprocessing
